# Import packages

In [ ]:
import folium
import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
from hmmlearn import hmm
from ipyleaflet import GeoData, LayersControl, Map
from IPython.display import HTML, display
from ipywidgets import HTML
from sklearn.preprocessing import StandardScaler

# Load data:

In [ ]:
spain_map = gpd.read_file("../data/maps/spain_map.geojson")

In [ ]:
olive_yield_df = pd.read_csv("../data/yearly_olive_data.csv")

In [ ]:
spei_df = pd.read_csv("../data/spei.csv", parse_dates=["date"])

In [ ]:
olive_yield_df.head()

In [ ]:
spei_df.head()

In [ ]:
s1 = set(olive_yield_df["province"])

In [ ]:
s2 = set(spei_df["province"])

In [ ]:
s1 - s2

In [ ]:
s2 - s1

In [ ]:
spei_df = spei_df[spei_df["province"].isin(olive_yield_df["province"])]

In [ ]:
spei_df.head()

In [ ]:
olive_yield_df.head()

In [ ]:
start_year = 2000
end_year = 2021

In [ ]:
olive_yield_df = olive_yield_df[olive_yield_df["year"].between(start_year, end_year)]

In [ ]:
spei_df = spei_df[spei_df["date"].dt.year.between(start_year, end_year)]

In [ ]:
olive_yield_df["year"].unique()

In [ ]:
spei_df["date"].dt.year.unique()

In [ ]:
olive_yield_df.head()

In [ ]:
spei_df.head()

# High producing provinces:

In [ ]:
top_5_provinces = (
    olive_yield_df.groupby(by="province", as_index=False)
    .agg(total=pd.NamedAgg(column="total", aggfunc="sum"))
    .sort_values(by="total", ascending=False)
    .reset_index()
    .loc[:5, "province"]
    .tolist()
)

In [ ]:
print(top_5_provinces)

In [ ]:
fig = px.line(
    data_frame=olive_yield_df[olive_yield_df["province"].isin(top_5_provinces)],
    x="year",
    y="total",
    color="province",
)

fig.show()

# Hidden Markov Model

In [ ]:
spei_df["year"] = spei_df["date"].dt.year
spei_df["month"] = spei_df["date"].dt.month

In [ ]:
merged_data = (
    pd.merge(spei_df, olive_yield_df, on=["province", "year"], how="left")
    .fillna({"total": 0, "table_olive": 0, "olive_oil": 0})
    .drop(
        columns=[
            "avg_pet",
            "avg_precipitation",
            "water_balance",
            "year",
            "month",
            "table_olive",
            "olive_oil",
        ]
    )
)

In [ ]:
merged_data.head()

In [ ]:
scaler_spei = StandardScaler()
scaler_yield = StandardScaler()

In [ ]:
merged_data["spei_scaled"] = scaler_spei.fit_transform(merged_data[["spei"]])

In [ ]:
merged_data["yield_scaled"] = scaler_yield.fit_transform(merged_data[["total"]])

In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

Number of years * number of months * number of provinces: `22 * 12 * 50 = 13200`

In [ ]:
state_labels = ["Low Drought Impact", "Medium Drought Impact", "High Drought Impact"]
n_states = len(state_labels)

In [ ]:
observations = {}
for province in merged_data["province"].unique():
    province_data = merged_data[merged_data["province"] == province]
    X = province_data[["spei_scaled", "yield_scaled"]].values
    observations[province] = X

In [ ]:
models = {}
for province, X in observations.items():
    model = hmm.GaussianHMM(
        n_components=n_states, covariance_type="full", n_iter=100000, random_state=42
    )
    model.fit(X)
    models[province] = model

    # Predict the hidden states
    hidden_states = model.predict(X)
    merged_data.loc[merged_data["province"] == province, "hidden_state"] = hidden_states

In [ ]:
state_mapping = {i: label for i, label in enumerate(state_labels)}
merged_data["state_label"] = merged_data["hidden_state"].map(state_mapping)

In [ ]:
merged_data.head()

In [ ]:
province_drought_impact_df = merged_data.groupby(by="province", as_index=False).agg(
    drought_impact=pd.NamedAgg(column="state_label", aggfunc="first")
)

In [ ]:
drought_colors_mapping = {
    "Low Drought Impact": "green",
    "Medium Drought Impact": "yellow",
    "High Drought Impact": "red",
}

In [ ]:
drought_impact_map = spain_map.merge(
    right=province_drought_impact_df, on="province"
).pipe(
    lambda df_: df_.assign(
        drought_color=df_["drought_impact"].map(drought_colors_mapping)
    )
)

In [ ]:
avg_spei_map = spain_map.merge(
    right=merged_data.groupby(by="province", as_index=False).agg(
        avg_spei=pd.NamedAgg(column="spei", aggfunc="mean")
    ),
    on="province",
)

In [ ]:
avg_yield_map = spain_map.merge(
    right=olive_yield_df.groupby(by="province", as_index=False).agg(
        avg_total=pd.NamedAgg(column="total", aggfunc="mean")
    ),
    on="province",
)

In [ ]:
map1 = drought_impact_map.explore(
    color="drought_color", style_kwds={"stroke": False}, height=400, width=400
)
map2 = avg_spei_map.explore(column="avg_spei", height=400, width=400)
map3 = avg_yield_map.explore(column="avg_total", height=400, width=400)

In [ ]:
html_string = """
<div style="display: flex; justify-content: space-between;">
    <div style="width: 33%;">{}</div>
    <div style="width: 33%;">{}</div>
    <div style="width: 33%;">{}</div>
</div>
""".format(
    map1._repr_html_(), map2._repr_html_(), map3._repr_html_()
)

In [ ]:
display(HTML(html_string))